<a href="https://colab.research.google.com/github/VeroFrias93/TOPICO-DE-LA-INDUSTRIA-1/blob/main/Practica_4_VMFP_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 4

**Nombre:** VERONICA MARISELA FRIAS PERALTA

**e-mail:** veronica.frias6148@alumnos.udg.mx

##MODULES

In [1]:
import panel as pn
import panel.widgets as pnw
import param
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

pn.extension('plotly')

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import math
from math import pi

import holoviews as hv
pn.extension(sizing_mode = 'stretch_width')

from shapely.geometry import Point, LineString

##CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

##FUNCTIONS

In [3]:
###############################################################################################
# Brownian Motion Trajectory
###############################################################################################
def bm_2d(n_steps=1000, speed=6, s_x_pos=0, s_y_pos=0):
    """
    Arguments:
        n_steps:
        speed:
        s_pos:
    Returns:
        BM_2d_df:
    """
    # Init velocity vector
    velocity = Vec2d(speed,0)

    BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_x_pos, 'y_pos': s_y_pos}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)


    for i in range(n_steps-1):
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    # return dataframe
    return BM_2d_df

In [32]:
def crwv1(pasos: int = 200, velocidad: float = 4, posicion_inicial: list = [0, 0], exp_crw: float = 0.7) -> pd.DataFrame:
  """
Caminata Aleatoria Correlacionada (CRW) en 2 dimensiones.

  Argumentos:
      pasos (int): Número de pasos a simular.
      velocidad (float): Velocidad de la CRW.
      posicion_inicial (list): Posición inicial (x, y).
      exp_crw (float): Exponente de la distribución de Cauchy.

  Retorna:
      df_crw (pd.DataFrame): Trayectoria de la CRW con las columnas "x_pos" e "y_pos".
  """

  # Move velocidad definition outside the loop
  velocidad_actual = np.array([velocidad, 0])
  posicion_inicial = np.array(posicion_inicial)
  df_crw = pd.DataFrame(columns=["x_pos", "y_pos"])

  bm_2d = np.random.uniform(low=-np.pi, high=np.pi, size=pasos)

  for i in range(pasos):
      angulo_actual = bm_2d[i]
      cambio_angulo = angulo_actual
      velocidad_actual = velocidad_actual * np.array([np.cos(cambio_angulo), np.sin(cambio_angulo)])

      nueva_posicion = posicion_inicial + velocidad_actual * i
      df_crw = df_crw.append({"x_pos": nueva_posicion[0], "y_pos": nueva_posicion[1]}, ignore_index=True)

  return df_crw


  # Vuelo de Levy
def lvfv1(pasos=200, velocidad=4, s_pos=[0,0], cauchy_lv=0.4, exp_lv = 1):
    """
    Argumentos:
        pasos: Número de pasos.
        velocidad: Velocidad.
        posicion_inicial: Posición inicial.
        cauchy_lv: Coeficiente Cauchy para el Vuelo de Levy.
        exp_lv: Exponente del Vuelo de Levy.
    Return:
        df_lft (pd.DataFrame): Trayectoria del Vuelo de Lévy con las columnas "x_pos" e "y_pos"
    """



    velocidad = Vec2d(velocidad, 0)
    std_mstps = 3
    df_vuelo_levy = pd.DataFrame(columns=["x_pos", "y_pos"])
    temp_df = pd.DataFrame([{"x_pos": 0, "y_pos": 0}])
    df_vuelo_levy = pd.concat([df_vuelo_levy, temp_df], ignore_index=True)
    i = 1
    while i < pasos:
        angulo_giro = wrapcauchy.rvs(c=cauchy_lv, loc=0)
        velocidad = velocidad.rotated(angulo_giro)
        n_pasos = abs(math.floor(levy_stable.rvs(alpha=exp_lv, beta=1, loc=std_mstps)))
        if (i + n_pasos > pasos):
            n_pasos = pasos - 1

        c_paso = 0
        while c_paso < n_pasos:
            temp_df = pd.DataFrame([{"x_pos": df_vuelo_levy.x_pos[i-1] + velocidad.x,
                                     "y_pos": df_vuelo_levy.y_pos[i-1] + velocidad.y}])
            df_vuelo_levy = pd.concat([df_vuelo_levy, temp_df], ignore_index=True)
            c_paso += 1
            i += 1

    return df_vuelo_levy




In [5]:
n_steps=pnw.IntInput(name='Number of steps',value=200,step=2,start=0,end=2000)
speed = pnw.IntInput(name='Speed', value=8,step=1,start=0,end=10)
x_pos = pnw.FloatInput(name='Starting pos_x',value=0,step=1,start=0,end=1000)
y_pos = pnw.FloatInput(name='starting pos_y',value=0,step=1,start=0,end=1000)
crw_exp =pnw.FloatInput(name='Cauchy coefficient',value=0.6, step=0.2, start=0.5, end=1)
lv_exp = pnw.FloatInput(name='Levy exponent',value=0.6,step=0.6,start=0.5,end=1)


##Trajectories

In [8]:
# BM
@pn.depends(n_steps, speed, x_pos, y_pos)
def plot_traj(n_steps, speed, x_pos, y_pos):
    fig_traj_bm = go.Figure()
    bm_df = trayectoria_browniana_2d(n_steps,speed,[x_pos, y_pos])

    fig_traj_bm.add_trace(
        go.Scatter3d(x=bm_df.x_pos,
                     y=bm_df.y_pos,
                     z=bm_df.index,
                     marker=dict(size=2),
                     line=dict(color='pink', width=2),
                     mode='lines',
                     name= f'BM={n_steps}',

                     showlegend=True))

    fig_traj_bm.update_traces(groupnorm='fraction',
               selector=dict(type='scatter'))
    return fig_traj_bm


    # CRW
@pn.depends(n_steps, speed, x_pos, y_pos, crw_exp)
def plot_traj_crw(n_steps, speed, x_pos, y_pos, crw_exp):
    fig_traj_crw = go.Figure()
    crw_df = crwv1(n_steps=n_steps, speed=speed, s_pos=[x_pos, y_pos], crw_exp=crw_exp)

    fig_traj_crw.add_trace(
        go.Scatter3d(x=crw_df.x_pos,
                     y=crw_df.y_pos,
                     z=crw_df.index,
                     marker=dict(size=2),
                     line=dict(color='pink', width=2),
                     mode='lines',
                     name= f'CRW= {crw_exp}',
                     showlegend=True))

    fig_traj_crw.update_traces(groupnorm='fraction',
                            selector=dict(type='scatter'))
    return fig_traj_crw

    # LVF
@pn.depends(n_steps, speed, x_pos, y_pos, crw_exp, lv_exp)
def plot_traj_lv(n_steps, speed, x_pos, y_pos, crw_exp, lv_exp):
    fig_traj_lv = go.Figure()
    lv_df = lvfv1(n_steps,speed, s_pos=[x_pos, y_pos], lvf_cauchy=crw_exp, lvf_exp=lv_exp)

    fig_traj_lv.add_trace(
        go.Scatter3d(x=lv_df.x_pos,
                     y=lv_df.y_pos,
                     z=lv_df.index,
                     marker=dict(size=2),
                     line=dict(color='pink', width=2),
                     mode='lines',
                     name= f'LVF= {lv_exp}',
                     showlegend=True))

    fig_traj_lv.update_traces(groupnorm='fraction',
                            selector=dict(type='scatter'))
    return fig_traj_lv


    # Path Length BM
@pn.depends(n_steps, speed, x_pos, y_pos)
def met_bm(n_steps, speed, x_pos, y_pos):
    fig_met_bm = go.Figure()
    bm_df = trayectoria_browniana_2d(n_steps, speed, [x_pos, y_pos])
    met_df = ([np.linalg.norm(bm_df.iloc[i-1]-bm_df.iloc[i]) for i in range(1, bm_df.shape[0])])
    pl_bm = np.cumsum(met_df)
    fig_met_bm.add_trace(go.Scatter(
        x=np.arange(len(pl_bm)),
        y=pl_bm,
        marker = dict(size=2),
        line = dict(color='yellow', width=2),
        mode = 'lines',
        name = 'Path Length',
        showlegend = True))
    fig_met_bm.update_layout(title_text='Path Length')

    return fig_met_bm

# Path Length CRW
@pn.depends(n_steps, speed, x_pos, y_pos, crw_exp)
def met_crw(n_steps, speed, x_pos, y_pos, crw_exp):
    fig_met_crw = go.Figure()
    crw_df = crwv1(n_steps=n_steps, speed=speed, s_pos=[x_pos, y_pos], crw_exp=crw_exp)

    met_df = ([np.linalg.norm(crw_df.iloc[i-1]-crw_df.iloc[i]) for i in range(1, crw_df.shape[0])])
    pl_crw = np.cumsum(met_df)
    fig_met_crw.add_trace(go.Scatter(
        x=np.arange(len(pl_crw)),
        y=pl_crw,
        marker = dict(size=2),
        line = dict(color='blue', width=2),
        mode = 'lines',
        name = 'Path Length',
        showlegend = True))
    fig_met_crw.update_layout(title_text='Path Length')

    return fig_met_crw


# Path Length LVF
@pn.depends(n_steps, speed, x_pos, y_pos, crw_exp, lv_exp)
def met_lv(n_steps, speed, x_pos, y_pos, crw_exp, lv_exp):
    fig_met_lv = go.Figure()
    lv_df = lvfv1(n_steps,speed, s_pos=[x_pos, y_pos], lvf_cauchy=crw_exp, lvf_exp=lv_exp)

    met_df = ([np.linalg.norm(lv_df.iloc[i-1]-lv_df.iloc[i]) for i in range(1, lv_df.shape[0])])
    pl_lv = np.cumsum(met_df)
    fig_met_lv.add_trace(go.Scatter(
        x=np.arange(len(pl_lv)),
        y=pl_lv,
        marker = dict(size=2),
        line = dict(color='red', width=2),
        mode = 'lines',
        name = 'Path Length',
        showlegend = True))

    fig_met_lv.update_layout(title_text='Path Length')

    return fig_met_lv

# MSD BM
@pn.depends(n_steps, speed, x_pos, y_pos)
def plot_bm_msd(n_steps, speed, x_pos, y_pos):
    msd_bm = np.empty(shape=(0))
    fig_met_bm_msd = go.Figure()
    bm_df = trayectoria_browniana_2d(n_steps, speed, [x_pos, y_pos])

    for tau in range(1, bm_df.shape[0]):
        met_df = ([(np.linalg.norm(bm_df.iloc[i-tau]-bm_df.iloc[i]))**2
                 for i in range(tau, bm_df.shape[0],1)])
        d = np.mean(met_df)
        msd_bm = np.append(msd_bm, d)
        msd_bm_df = pd.DataFrame()
        msd_bm_df['msd_bm'] = msd_bm

    fig_met_bm_msd.add_trace(go.Scatter(
        x=np.arange(len(msd_bm_df.index)),
        y=msd_bm_df.msd_bm,
        marker = dict(size=2),
        line = dict(color='yellow', width=2),
        mode = 'lines',
        name = 'Msd',
        showlegend = True))
    return fig_met_bm_msd

# MSD CRW
@pn.depends(n_steps, speed, x_pos, y_pos, crw_exp)
def plot_crw_msd(n_steps, speed, x_pos, y_pos, crw_exp):
    msd_crw = np.empty(shape=(0))
    fig_met_crw_msd = go.Figure()
    crw_df = crwv1(n_steps=n_steps, speed=speed, s_pos=[x_pos, y_pos], crw_exp=crw_exp)

    for tau in range(1, crw_df.shape[0]):
        met_df = ([(np.linalg.norm(crw_df.iloc[i-tau]-crw_df.iloc[i]))**2
                 for i in range(tau, crw_df.shape[0],1)])
        d = np.mean(met_df)
        msd_crw = np.append(msd_crw, d)
        msd_crw_df = pd.DataFrame()
        msd_crw_df['msd_crw'] = msd_crw

    fig_met_crw_msd.add_trace(go.Scatter(
        x=np.arange(len(msd_crw_df.index)),
        y=msd_crw_df.msd_crw,
        marker = dict(size=2),
        line = dict(color='yellow', width=2),
        mode = 'lines',
        name = 'Msd',
        showlegend = True))
    return fig_met_crw_msd

# MSD LVF
@pn.depends(n_steps, speed, x_pos, y_pos, crw_exp, lv_exp)
def plot_lvf_msd(n_steps, speed, x_pos, y_pos, crw_exp, lv_exp):
    msd_lv = np.empty(shape=(0))
    fig_met_lvf_msd = go.Figure()
    lvf_df = lvfv1(n_steps,speed,[x_pos, y_pos],crw_exp, lv_exp)

    for tau in range(1, lvf_df.shape[0]):
        met_df = ([(np.linalg.norm(lvf_df.iloc[i-tau]-lvf_df.iloc[i]))**2
                 for i in range(tau, lvf_df.shape[0],1)])
        d = np.mean(met_df)
        msd_lv = np.append(msd_lv, d)
        msd_lv_df = pd.DataFrame()
        msd_lv_df['msd_lvf'] = msd_lv

    fig_met_lvf_msd.add_trace(go.Scatter(
        x=np.arange(len(msd_lv_df.index)),
        y=msd_lv_df.msd_lvf,
        marker = dict(size=2),
        line = dict(color='yellow', width=2),
        mode = 'lines',
        name = 'Msd',
        showlegend = True))
    return fig_met_lvf_msd


In [35]:
# Correlated Random Walk 2D Function

def CRW_2d(pasos=1000, velocity=6, posIniX=0, posIniY=0, factorCauchy=0.4,):
  velocity = Vec2d(velocity, 0)
  CRW_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_pos', 'msd', 'pl'])
  temp_df = pd.DataFrame([{'x_pos':posIniX, 'y_pos':posIniY, 'z_pos':0, 'msd':0, 'pl':0}])
  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index = True)
  ListaPDTrayectorias = []
  distribucionDireccion = wrapcauchy(factorCauchy, 0, scale = 1)
  listaDirecciones = distribucionDireccion.rvs(size = pasos)

  for i in range(0, pasos -1):
    turn_angle = listaDirecciones[i]
    velocity = velocity.rotated(turn_angle)
    temp_df = pd.DataFrame([{'x_pos':CRW_2d_df.x_pos[i]+velocity.x, 'y_pos':CRW_2d_df.y_pos[i]+velocity.y, 'z_pos':i, 'msd':0, 'pl':0}])
    CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index = True)

  return CRW_2d_df

In [38]:
# Levy Flight Function

def LevyFlyPd(pasos,speed, miu, levyExponent, levyBeta, factorCauchy, posIniX=0, posIniY=0):
  s_pos = [0,0]
  velocity = Vec2d(speed,0)

  distribucionDireccion = wrapcauchy(0.4, 0, scale = 1)
  listaDirecciones = distribucionDireccion.rvs(size = pasos)
  distribucionPasos = levy_stable(alpha = levyExponent, beta = levyBeta, loc = miu)
  listaCantidadPasos = distribucionPasos.rvs(size = pasos)

  # Creamos el DataFrame con Pandas
  pdLevy = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_pos', 'msd', 'pl'])
  temp3d = pd.DataFrame([{'x_pos':posIniX, 'y_pos':posIniY, 'z_pos':0, 'msd':0, 'pl':0}])
  pdLevy = pd.concat([pdLevy, temp3d], ignore_index = True)

  PasosPendientes=0
  IndicePasos = 0
  IndiceCambiosDireccion = 0

  for i in range(0, pasos-1):
    if PasosPendientes < 1:
      velocity = velocity.rotated(listaDirecciones[IndiceCambiosDireccion])
      IndiceCambiosDireccion = IndiceCambiosDireccion + 1
      temp3d = pd.DataFrame([{'x_pos':pdLevy.x_pos[i]+velocity.x, 'y_pos':pdLevy.y_pos[i]+velocity.y, 'z_pos':i, 'msd':0, 'pl':0}])
      pdLevy = pd.concat([pdLevy, temp3d], ignore_index = True)
      PasosPendientes = int(listaCantidadPasos[IndicePasos])
      IndicePasos = IndicePasos+1
    else:
      temp3d = pd.DataFrame([{'x_pos':pdLevy.x_pos[i]+velocity.x, 'y_pos':pdLevy.y_pos[i]+velocity.y, 'z_pos':i, 'msd':0, 'pl':0}])
      pdLevy = pd.concat([pdLevy, temp3d], ignore_index = True)
      PasosPendientes = PasosPendientes-1

    return pdLevy

In [40]:
# Función dinámica para generar métricas con base a parámetros

def GraficaMetrica(trayectoria, metrica=1, comentario = 'BM Walk MSD'):
  GraficaMetrica = go.Figure()

  if metrica == 1:
    GraficaMetrica.add_trace(go.Scatter(
      x = trayectoria.z_pos,
      y = trayectoria.msd,
      marker = dict(size = 3),
      line = dict(width = 1),
      mode = 'lines',
      name = comentario,
      showlegend = True
    ))
  else:
    GraficaMetrica.add_trace(go.Scatter(
        x = trayectoria.z_pos,
        y = trayectoria.pl,
        marker = dict(size = 3),
        line = dict(width = 1),
        mode = 'lines',
        name = comentario,
        showlegend = True
    ))

  return GraficaMetrica

In [9]:
pn.Column(plot_lvf_msd)

Column(sizing_mode='stretch_width')
    [0] ParamFunction(function, _pane=Plotly, defer_load=False, sizing_mode='stretch_width')

#DASHBOARD

In [48]:

# Definir los paneles para la visualización de trayectoria y métricas
trayectoria_pane = pn.pane.Plotly()
metrica_pane = pn.pane.Markdown()
trayectoria_3d_pane = pn.pane.Plotly()

# Definir los selectores para elegir la métrica y el tipo de trayectoria
metrica_selector = pnw.RadioButtonGroup(options=['Path Length', 'Mean Squared Displacement'], value='Path Length', name='Métrica')
trayectoria_selector = pnw.RadioButtonGroup(options=['Movimiento Browniano', 'Caminata Aleatoria Correlacionada', 'Vuelo de Levy'], value='Movimiento Browniano', name='Tipo de Trayectoria')
trayectoria_selector = pnw.RadioButtonGroup(name='Tipo de Trayectoria', options=['Movimiento Browniano', 'Caminata Aleatoria Correlacionada', 'Vuelo de Levy'])

# Definir las funciones para actualizar la visualización y métrica
def actualizar_trayectoria(event):
    tipo_trayectoria = trayectoria_selector.value
    if tipo_trayectoria == 'Movimiento Browniano':
        trayectoria_df = trayectoria_browniana_2d(n_steps.value, speed.value, [x_pos.value, y_pos.value])
    elif tipo_trayectoria == 'Caminata Aleatoria Correlacionada':
        trayectoria_df = crwv1(n_steps.value, speed.value, [x_pos.value, y_pos.value], crw_exp.value)
    elif tipo_trayectoria == 'Vuelo de Levy':
        trayectoria_df = lvfv1(n_steps.value, speed.value, [x_pos.value, y_pos.value], crw_exp.value, lv_exp.value)

    fig = go.Figure(data=go.Scatter(x=trayectoria_df['x_pos'], y=trayectoria_df['y_pos'], mode='lines'))
    fig.update_layout(title=f'Trayectoria - {tipo_trayectoria}', xaxis_title='X', yaxis_title='Y')
    trayectoria_pane.object = fig

def actualizar_metrica(event):
    tipo_metrica = metrica_selector.value
    tipo_trayectoria = trayectoria_selector.value
    if tipo_trayectoria == 'Movimiento Browniano':
        trayectoria_df = trayectoria_browniana_2d(n_steps.value, speed.value, [x_pos.value, y_pos.value])
    elif tipo_trayectoria == 'Caminata Aleatoria Correlacionada':
        trayectoria_df = crwv1(n_steps.value, speed.value, [x_pos.value, y_pos.value], crw_exp.value)
    elif tipo_trayectoria == 'Vuelo de Levy':
        trayectoria_df = lvfv1(n_steps.value, speed.value, [x_pos.value, y_pos.value], crw_exp.value, lv_exp.value)

    if tipo_metrica == 'Path Length':
        distancia = np.linalg.norm(trayectoria_df[['x_pos', 'y_pos']].diff().dropna(), axis=1)
        valor_metrica = np.sum(distancia)
    elif tipo_metrica == 'Mean Squared Displacement':
        desplazamiento = np.linalg.norm(trayectoria_df[['x_pos', 'y_pos']].diff().dropna(), axis=1)**2
        valor_metrica = np.mean(desplazamiento)

    metrica_pane.object = f'Valor de la métrica {tipo_metrica}: {valor_metrica}'


def actualizar_trayectoria_3d(event):
    tipo_trayectoria = trayectoria_selector.value
    if tipo_trayectoria == 'Movimiento Browniano':
        trayectoria_df = trayectoria_browniana_2d(n_steps.value, speed.value, [x_pos.value, y_pos.value])
    elif tipo_trayectoria == 'Caminata Aleatoria Correlacionada':
        trayectoria_df = crwv1(n_steps.value, speed.value, [x_pos.value, y_pos.value], crw_exp.value)
    elif tipo_trayectoria == 'Vuelo de Levy':
        trayectoria_df = lvfv1(n_steps.value, speed.value, [x_pos.value, y_pos.value], crw_exp.value, lv_exp.value)

    fig = go.Figure(data=[go.Scatter3d(x=trayectoria_df['x_pos'], y=trayectoria_df['y_pos'], z=trayectoria_df.index, mode='lines')])
    fig.update_layout(title=f'Trayectoria 3D - {tipo_trayectoria}', scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Paso'))
    trayectoria_3d_pane.object = fig

 # Definir una función para manejar los cambios en el tipo de trayectoria seleccionada
def actualizar_tipo_trayectoria(event):
    tipo_trayectoria = event.new
    if tipo_trayectoria == 'Movimiento Browniano':
        # Ocultar widgets específicos de otras trayectorias y mostrar los de Movimiento Browniano
        crw_exp_panel.visible = False
        lv_exp_panel.visible = False
    elif tipo_trayectoria == 'Caminata Aleatoria Correlacionada':
        # Mostrar el widget para el coeficiente Cauchy y ocultar el de Vuelo de Levy
        crw_exp_panel.visible = True
        lv_exp_panel.visible = False
    elif tipo_trayectoria == 'Vuelo de Levy':
        # Mostrar el widget para el coeficiente Cauchy y el de Vuelo de Levy
        crw_exp_panel.visible = True
        lv_exp_panel.visible = True

# Definir el widget para el coeficiente Cauchy
crw_exp_panel = pn.Column(pnw.FloatInput(name='Cauchy coefficient', value=0.6, step=0.1, start=0.1, end=1))
# Definir el widget para el exponente del Vuelo de Levy
lv_exp_panel = pn.Column(pnw.FloatInput(name='Levy exponent', value=0.4, step=0.1, start=0.1, end=1))

# Definir una función para manejar los cambios en el tipo de trayectoria seleccionada
def actualizar_tipo_trayectoria(event):
    tipo_trayectoria = event.new
    if tipo_trayectoria == 'Movimiento Browniano':
        # Ocultar widgets específicos de otras trayectorias y mostrar los de Movimiento Browniano
        crw_exp_panel.visible = False
        lv_exp_panel.visible = False
    elif tipo_trayectoria == 'Caminata Aleatoria Correlacionada':
        # Ocultar widgets específicos de otras trayectorias y mostrar los de Caminata Aleatoria Correlacionada
        crw_exp_panel.visible = True
        lv_exp_panel.visible = False
    elif tipo_trayectoria == 'Vuelo de Levy':
        # Ocultar widgets específicos de otras trayectorias y mostrar los de Vuelo de Levy
        crw_exp_panel.visible = True
        lv_exp_panel.visible = True



IndentationError: unexpected indent (<ipython-input-48-9efe86ef2e00>, line 21)